In [1]:
from room import Room
from player import Player
from world import World

import random
from ast import literal_eval

# Load world
world = World()


# You may uncomment the smaller graphs for development and testing purposes.
# map_file = "maps/test_line.txt"
# map_file = "maps/test_cross.txt"
# map_file = "maps/test_loop.txt"
map_file = "maps/test_loop_fork.txt"
# map_file = "maps/main_maze.txt"

# Loads the map into a dictionary
room_graph=literal_eval(open(map_file, "r").read())
world.load_graph(room_graph)

In [2]:
# Print an ASCII map
world.print_rooms()

#####
#                                        #
#      017       002       014           #
#       |         |         |            #
#       |         |         |            #
#      016--015--001--012--013           #
#                 |                      #
#                 |                      #
#      008--007--000--003--004           #
#       |         |                      #
#       |         |                      #
#      009       005                     #
#       |         |                      #
#       |         |                      #
#      010--011--006                     #
#                                        #

#####


In [3]:
def get_connected():
    global player
    exits = player.current_room.get_exits()
    exit_ids = [player.current_room.get_room_in_direction(direction).id for direction in player.current_room.get_exits()]
    num_exits = len(exits)
    return num_exits, exit_ids, exits

In [4]:
# init
player = Player(world.starting_room)
memory = dict()
travel_log = []
start_loc = player.current_room.id
memory[start_loc] = ["Start", "Gray"]
memory

{0: ['Start', 'Gray']}

In [5]:
get_connected()

(4, [1, 5, 7, 3], ['n', 's', 'w', 'e'])

# Begin Travel function

In [6]:
print(get_connected())
print(player.current_room.id)
memory

(4, [1, 5, 7, 3], ['n', 's', 'w', 'e'])
0


{0: ['Start', 'Gray']}

In [7]:
num_exits, exit_ids, exits = get_connected()
for num in range(num_exits):
    if exit_ids[num] not in memory:
        memory[exit_ids[num]] = [player.current_room.id, "White"]
mem_filter = {k:v for (k,v) in memory.items() if (k in exit_ids)&(v[1]=="White")}

In [8]:
# print(get_connected())
# print(player.current_room.id)
# print(memory)
# list(mem_filter.keys())[0]

In [9]:
# for (k,v) in memory.items():
#     if v[1] == "White":
#         print(k,v)

In [10]:
def travel(travel_log):
    #1. Make sure connections to current node are in memory
    num_exits, exit_ids, exits = get_connected()
    for num in range(num_exits):
        if exit_ids[num] not in memory:
            memory[exit_ids[num]] = [player.current_room.id, "White"]

    #2. if unexplored Nodes than travel them
    mem_filter = {k:v for (k,v) in memory.items() if (k in exit_ids)&(v[1]=="White")}
    if len(mem_filter) > 0:
        #2.1 mark the node gray before leaving
        memory[player.current_room.id][1] = "Gray"
        #2.2 get the node we are traveling too
        next_node = list(mem_filter.keys())[0]
        #2.3 get the direction of the node
        for num in range(num_exits):
            if exit_ids[num] == next_node:
                dir_next_node = exits[num]
        #2.4 travel to next node
        player.travel(exits[num])
        #2.5 add move to travel log
        travel_log.append(dir_next_node)
        #2.6 Start recursion
        travel(travel_log)

    #3. if no unexplored nodes return to parent node
    elif len(mem_filter) == 0:

        #3.1 check if has parent node or is start location
        if memory[player.current_room.id][0] != "Start":
            #3.1.1 mark the node gray before leaving
            memory[player.current_room.id][1] = "Black"
            #3.1.2 get the node we are traveling too
            next_node = memory[player.current_room.id][0]
            #3.1.3 get the direction of the node
            for num in range(num_exits):
                if exit_ids[num] == next_node:
                    dir_next_node = exits[num]
            #3.1.4 travel to next node
            player.travel(exits[num])
            #3.1.5 add move to travel log
            travel_log.append(dir_next_node)
            #3.1.6 Start recursion
            travel(travel_log)

            #3.1.2 if start return travel_log
        else:
            return memory, travel_log

In [11]:
# def travel(travel_log):
#     # When Entering a new room I should check all of its connections and ensure they are in memory
#     # Also should set current room to gray
#     memory[player.current_room.id][1] = "Gray"
#     num_exits, exit_ids, exits = get_connected()
#     for num in range(num_exits):
#         if exit_ids[num] not in memory:
#             memory[exit_ids[num]] = [player.current_room.id, "White"]

#     # Determine where to travel Next
#     for num in range(num_exits):
#         if memory[exit_ids[num]][1] == "White":
#             travel_log.append(exits[num])
#             player.travel(exits[num])
#             travel(travel_log)
    
#     # After traveling all paths leading to this Node we can back track to the start location
#     # First Double check that all nodes connected to current node are not white
#     white_count = 0
#     for num in range(num_exits):
#         if memory[exit_ids[num]][1] == "White":
#             white_count += 1
#     # Then find which direction leads to the parent node unless you are at the starting point
#     if (white_count == 0) & (player.current_room.id != 0):
#         parent_node = memory[player.current_room.id][0]
#         for num in range(num_exits):
#             if exit_ids[num] == parent_node:
#                 memory[player.current_room.id][1] = "Black"
#                 travel_log.append(exits[num])
#                 player.travel(exits[num])
#     else:
#         memory[player.current_room.id][1] = "Black"

#     return memory, travel_log

In [12]:
memory, traversal_path = travel(travel_log)

RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
memory

In [ ]:
print(traversal_path)

In [ ]:
#! ==================== DO NOT MODIFY ====================
# TRAVERSAL TEST - DO NOT MODIFY
visited_rooms = set()
player.current_room = world.starting_room
visited_rooms.add(player.current_room)

for move in traversal_path:
    player.travel(move)
    visited_rooms.add(player.current_room)

if len(visited_rooms) == len(room_graph):
    print(f"TESTS PASSED: {len(traversal_path)} moves, {len(visited_rooms)} rooms visited")
else:
    print("TESTS FAILED: INCOMPLETE TRAVERSAL")
    print(f"{len(room_graph) - len(visited_rooms)} unvisited rooms")
#! ==================== DO NOT MODIFY ====================